In [29]:
# Set up API_KEY to access OPENAI models, Langchain ecosystem

import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
    os.environ["LANGCHAIN_TRACING_V2"] = "true"


    

In [30]:
#Connect chinook db through sqlite for querying the underlying tables

from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook_db.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")
db

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [31]:
#Verify the tables & data can be accessed

print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']

CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24)

In [32]:
!pip install --upgrade langsmith

REquirement already satisfied: langsmith in c:\users\sures\final_projects\genai\sql_chatbot\env\lib\site-packages (0.2.10)


In [33]:


#Invoke the openai chat model 

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")
llm


ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002943CC4E210>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002943F5056A0>, root_client=<openai.OpenAI object at 0x000002943D610A10>, root_async_client=<openai.AsyncOpenAI object at 0x000002943F5074A0>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [34]:
!pip install --upgrade httpx langchain


In [35]:
# Generate the SQL query for the provided question

from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": "How many employees are there"})
print(query)


SELECT COUNT("EmployeeId") AS "TotalEmployees" FROM "Employee"


In [36]:
#Display the o/p of the above generated SQL query

from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
execute_query = QuerySQLDatabaseTool(db=db)
execute_query.invoke(query)


'[(8,)]'

In [37]:
# Convert the raw SQL o/p into human readable formatted text

from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
     """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

 Question: {question}
 SQL Query: {query}
 SQL Result: {result}
 Answer: """
 )

rephrase_answer = answer_prompt | llm | StrOutputParser()

chain = (
     RunnablePassthrough.assign(query=generate_query).assign(
         result=itemgetter("query") | execute_query
     )
     | rephrase_answer
 )

chain.invoke({"question": "How many employees are there"})


'There are 8 employees.'

In [38]:
#curate a set of examples that cover a broad range of query types and complexities.

examples = [
    {
        "input": "Count of customerID where total >20",
        "query": "select count(*) from Invoice where Total >20;"
    },
    {
        "input": "List out the customerID from Brazil",
        "query": "select CustomerId from Customer where country = 'Brazil';"
    },
    {
        "input": "Total number of Albhums",
        "query": "select count(*) from Album;"
    },
    {
        "input": "List out the firstName & LastName of the employees belongs to IT-Staff",
        "query": "select FirstName,LastName from Employee where Title = 'IT Staff';"
    },
    {
        "input": "How many number of Media types are available",
        "query": "select count(*) from MediaType;"
    },
    {
     'input':"List of customer's first name belongs to Germany",
     "query": "select  DISTINCT a.FirstName from Customer a, Invoice b where a.CustomerId = b.CustomerId and b.BillingCountry ='Germany';"
    }
]

In [39]:
#Creating a Few-Shot Learning Template

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
     [
         ("human", "{input}\nSQLQuery:"),
         ("ai", "{query}"),
     ]
 )
few_shot_prompt = FewShotChatMessagePromptTemplate(
     example_prompt=example_prompt,
     examples=examples,
     # input_variables=["input","top_k"],
     input_variables=["input"],
 )
print(few_shot_prompt.format(input1="How many employees are there?"))


Human: Count of customerID where total >20
SQLQuery:
AI: select count(*) from Invoice where Total >20;
Human: List out the customerID from Brazil
SQLQuery:
AI: select CustomerId from Customer where country = 'Brazil';
Human: Total number of Albhums
SQLQuery:
AI: select count(*) from Album;
Human: List out the firstName & LastName of the employees belongs to IT-Staff
SQLQuery:
AI: select FirstName,LastName from Employee where Title = 'IT Staff';
Human: How many number of Media types are available
SQLQuery:
AI: select count(*) from MediaType;
Human: List of customer's first name belongs to Germany
SQLQuery:
AI: select  DISTINCT a.FirstName from Customer a, Invoice b where a.CustomerId = b.CustomerId and b.BillingCountry ='Germany';


In [40]:
##Implementing Dynamic Few-Shot Selection

from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
     examples,
     OpenAIEmbeddings(),
     vectorstore,
     k=2,
     input_keys=["input"],
 )
example_selector.select_examples({"input": "How many employees are there?"})
few_shot_prompt = FewShotChatMessagePromptTemplate(
     example_prompt=example_prompt,
     example_selector=example_selector,
     input_variables=["input","top_k"],
 )
print(few_shot_prompt.format(input="How many employees are there?"))


Human: Total number of Albhums
SQLQuery:
AI: select count(*) from Album;
Human: List out the firstName & LastName of the employees belongs to IT-Staff
SQLQuery:
AI: select FirstName,LastName from Employee where Title = 'IT Staff';


In [41]:
final_prompt = ChatPromptTemplate.from_messages(
     [
         ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
         few_shot_prompt,
         ("human", "{input}"),
     ]
 )
print(final_prompt.format(input="How many employees are there?",table_info="some table info"))



System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries.
Human: Total number of Albhums
SQLQuery:
AI: select count(*) from Album;
Human: List out the firstName & LastName of the employees belongs to IT-Staff
SQLQuery:
AI: select FirstName,LastName from Employee where Title = 'IT Staff';
Human: How many employees are there?


In [42]:
# Integrating with LangChain to execute the genrated query based on dynamic few shot selectors

generate_query = create_sql_query_chain(llm, db,final_prompt)
chain = (
 RunnablePassthrough.assign(query=generate_query).assign(
     result=itemgetter("query") | execute_query
 )
 | rephrase_answer
 )
chain.invoke({"question": "How many IT staff are there ?"})

'There are 2 IT staff members.'

In [43]:
## Implementation of Dynamic Relevant Table Selection

from operator import itemgetter
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
import pandas as pd
from pydantic import BaseModel

def get_table_details():
    # Read the CSV file into a DataFrame
    table_description = pd.read_csv("database_table_descriptions.csv")
    table_docs = []

    # Iterate over the DataFrame rows to create Document objects
    table_details = ""
    for index, row in table_description.iterrows():
        table_details = table_details + "Table Name:" + row['Table'] + "\n" + "Table Description:" + row['Description'] + "\n\n"

    return table_details


class Table(BaseModel):
    """Table in SQL database."""

    name: str = Field(description="Name of table in SQL database.")

# table_names = "\n".join(db.get_usable_table_names())
table_details = get_table_details()
print(table_details)


c:\Users\sures\Final_projects\GenAI\SQL_chatbot\env\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


Table Name:Album
Table Description:Contains details about music albums, including their titles and the artists who created them. It connects to the Artist table via a foreign key.

Table Name:Artist
Table Description:Stores information about music artists, such as their names. Each artist can have multiple albums associated with them.

Table Name:Customer
Table Description:Represents customers of the digital music store, storing personal and contact information. It links to the Employee table for assigned support representatives.

Table Name:Employee
Table Description:Contains details about employees of the store, including their roles, reporting managers, and contact details. It supports self-referencing for management hierarchy.

Table Name:Genre
Table Description:Lists various music genres available in the store, serving as a reference for classifying tracks in the Track table.

Table Name:Invoice
Table Description:Tracks purchase transactions made by customers, including billing de

In [44]:
# Invoking  the Relevant Table selection class in LLM

table_details_prompt = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
The tables are:

{table_details}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)
tables = table_chain.invoke({"input": "give me details of customer,Invoice and their order count"})
tables


C:\Users\sures\AppData\Local\Temp\ipykernel_29260\960961279.py:8: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` thatis available on ChatModels capable of tool calling.You can read more about the method here: <https://python.langchain.com/docs/modules/model_io/chat/structured_output/>. Please follow our extraction use case documentation for more guidelineson how to do information extraction with LLMs.<https://python.langchain.com/docs/use_cases/extraction/>. with_structured_output does not currently support a list of pydantic schemas. If this is a blocker or if you notice other issues, please provide feedback here:<https://github.com/langchain-ai/langchain/discussions/18154>
  table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)
c:\Users\sures\Final_projects\GenAI\SQL_chatbot\env\Lib\site-packages\pydantic\json_schema.py:2279: PydanticJsonSchemaWarning: Default value default=PydanticUndefined des

[Table(name='Customer'), Table(name='Invoice'), Table(name='Invoice Line')]

In [45]:
def get_tables(tables: List[Table]) -> List[str]:
    tables  = [table.name for table in tables]
    return tables

select_table = {"input": itemgetter("question")} | create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt) | get_tables
select_table.invoke({"question": "give me details of customer, Invoice and their order count"})


c:\Users\sures\Final_projects\GenAI\SQL_chatbot\env\Lib\site-packages\pydantic\json_schema.py:2279: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='Name of table in SQL database.' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


['Customer', 'Invoice', 'Invoice Line']

In [46]:
 # Rephrase the generated output in a readable format

generate_query = create_sql_query_chain(llm, db,final_prompt)

chain = (
 RunnablePassthrough.assign(table_names_to_use=select_table) |
 RunnablePassthrough.assign(query=generate_query).assign(
     result=itemgetter("query") | execute_query
 )
 | rephrase_answer
 )


In [47]:
#Setting Up Message History

from langchain_community.chat_message_histories import ChatMessageHistory
history = ChatMessageHistory()


In [48]:
# leverages historical interactions/ chat messages

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
final_prompt = ChatPromptTemplate.from_messages(
     [
         ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and should be considered while answering follow up questions"),
         few_shot_prompt,
         MessagesPlaceholder(variable_name="messages"),
         ("human", "{input}"),
     ]
 )
print(final_prompt.format(input="How many tracks are there?",table_info="some table info",messages=[]))


System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and should be considered while answering follow up questions
Human: How many number of Media types are available
SQLQuery:
AI: select count(*) from MediaType;
Human: Total number of Albhums
SQLQuery:
AI: select count(*) from Album;
Human: How many tracks are there?


In [49]:
# Dynamically adapting the prompt  to use chat history messages

generate_query = create_sql_query_chain(llm, db,final_prompt)

 chain = (
 RunnablePassthrough.assign(table_names_to_use=select_table) |
 RunnablePassthrough.assign(query=generate_query).assign(
     result=itemgetter("query") | execute_query
 )
 | rephrase_answer
 )


In [50]:
question = "How many customers with order count more than 5"
response = chain.invoke({"question": question,"messages":history.messages})
response


'There are 59 customers with an order count of more than 5.'

In [51]:
#Add user & answers to the  message history

history.add_user_message(question)
history.add_ai_message(response)
history.messages


[HumanMessage(content='How many customers with order count more than 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='There are 59 customers with an order count of more than 5.', additional_kwargs={}, response_metadata={})]

In [52]:
response = chain.invoke({"question": "Can you list customers  names?","messages":history.messages, "table_details":table_details})
response

"Yes, here is a list of customer names:\n- Luís Gonçalves\n- Leonie Köhler\n- François Tremblay\n- Bjørn Hansen\n- František Wichterlová\n- Helena Holý\n- Astrid Gruber\n- Daan Peeters\n- Kara Nielsen\n- Eduardo Martins\n- Alexandre Rocha\n- Roberto Almeida\n- Fernanda Ramos\n- Mark Philips\n- Jennifer Peterson\n- Frank Harris\n- Jack Smith\n- Michelle Brooks\n- Tim Goyer\n- Dan Miller\n- Kathy Chase\n- Heather Leacock\n- John Gordon\n- Frank Ralston\n- Victor Stevens\n- Richard Cunningham\n- Patrick Gray\n- Julia Barnett\n- Robert Brown\n- Edward Francis\n- Martha Silk\n- Aaron Mitchell\n- Ellie Sullivan\n- João Fernandes\n- Madalena Sampaio\n- Hannah Schneider\n- Fynn Zimmermann\n- Niklas Schröder\n- Camille Bernard\n- Dominique Lefebvre\n- Marc Dubois\n- Wyatt Girard\n- Isabelle Mercier\n- Terhi Hämäläinen\n- Ladislav Kovács\n- Hugh O'Reilly\n- Lucas Mancini\n- Johannes Van der Berg\n- Stanisław Wójcik\n- Enrique Muñoz\n- Joakim Johansson\n- Emma Jones\n- Phil Hughes\n- Steve Murray